Mounting Google Drive and setting approapriate version of tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [ ]:
pip install nervaluate

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from tensorflow.contrib.crf import crf_log_likelihood, crf_decode
from tensorflow.contrib.rnn import LSTMCell, GRUCell, BasicRNNCell
from sklearn.metrics import f1_score, precision_score, recall_score
from nervaluate import Evaluator
import sys
import random
import time
import pickle

In [ ]:
class lstm_crf(object):

    '''
    lstm-crf for ner tagging
    
    parameters:
      - embedding_matrix: numpy array[float]
        numpy array of word embeddings
        each row should represent a word embedding
        NOTE: the word index 0 is dropped, so the first row is ignored
      - num_classes: int
        number of output classes
      - max_len: int (default: 50)
        maximum number of input tokens in any sequence
      - rnn_size: int (default: 300)
        number of rnn units in RNN layer
      - dropout_keep: float (default: 0.9)
        dropout keep rate after rnn layer
      - lr: float (default: 1E-4)
        learning rate for adam optimizer
       
    methods:
      - train(X,y,doc_lens,batch_size=128,epochs=25,patience=10,
              validation_data=None,savebest=False,filepath=None)
        train network on given data
      - predict(X,doc_lens,batch_size=128)
        return the predicted labels, flattened labels ignoring padding tokens, 
        and confidence scores for given data
      - score(X,y,doc_lens,batch_size=128)
        return the entity-level exact F1 score for given input sequences
      - save(filepath)
        save the model weights to a file
      - load(filepath)
        load model weights from a file
    '''

    def __init__(self,embedding_matrix,num_classes,max_len=50,rnn_size=300,dropout_keep=0.9,learning_rate=1E-4):
    
        #model params
        self.num_classes = num_classes
        self.embeddings = embedding_matrix.astype(np.float32)
        self.max_len = max_len
        self.rnn_size = rnn_size
        self.dropout_keep = dropout_keep
        self.idx2label = {0:'O',1:'B-ENT',2:'I-ENT'}
        self.lr = learning_rate
        
        #model inputs
        self.doc_inputs = tf.placeholder(tf.float32,shape=[None,max_len,200])
        self.doc_lens = tf.placeholder(tf.int32,shape=[None])
        self.labels = tf.placeholder(tf.int32,shape=[None,max_len])
        self.doc_idx = tf.placeholder(tf.int32,shape=[None,2])
        self.dropout = tf.placeholder(tf.float32)
        rnn_input = tf.nn.dropout(self.doc_inputs,self.dropout)
                      
        #bi-lstm
        with tf.variable_scope('rnn',initializer=tf.contrib.layers.xavier_initializer()):
            [outputs_fw,outputs_bw],_ = tf.nn.bidirectional_dynamic_rnn(
                            LSTMCell(self.rnn_size/2),LSTMCell(self.rnn_size/2),
                            rnn_input,sequence_length=self.doc_lens,dtype=tf.float32)
            outputs = tf.concat((outputs_fw,outputs_bw),2)
            outputs = tf.nn.dropout(outputs,self.dropout)
        
        #linear chain conditional random field
        unary_scores = tf.layers.dense(outputs,self.num_classes,
                       kernel_initializer=tf.contrib.layers.xavier_initializer())
        log_likelihood, self.transition_params = \
                       crf_log_likelihood(unary_scores,self.labels,self.doc_lens)
        self.pred, viterbi_score = crf_decode(unary_scores,self.transition_params,self.doc_lens)
        self.pred_flat = tf.gather_nd(self.pred,self.doc_idx)
        seq_score,_ = crf_log_likelihood(unary_scores,self.pred,self.doc_lens,
                                         self.transition_params)
        self.seq_score = seq_score/tf.cast(self.doc_lens,tf.float32)

        #loss, accuracy, and training functions
        self.loss = tf.reduce_mean(-log_likelihood)
        self.optimizer = tf.train.AdamOptimizer(self.lr,0.9,0.99).minimize(self.loss)
        
        #init ops
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        self.saver = tf.train.Saver()
        self.sess = tf.Session(config=config)
        self.sess.run(tf.global_variables_initializer())

    def _gen_doc_idx(self,doc_lens):
    
        doc_idx = []
        for i,l in enumerate(doc_lens):
            for j in range(l):
                doc_idx.append([i,j])
        doc_idx = np.array(doc_idx)

        return doc_idx
        
    def _flatten_y(self,y,doc_lens):
    
        y_flattened = []
        for i,(doc,l) in enumerate(zip(y,doc_lens)):       
            y_flattened.extend(doc[:l])
            
        return y_flattened
                    
    def train(self,X,y,doc_lens,batch_size=128,epochs=500,patience=20,
              validation_data=None,savebest=False,filepath=None):
    
        '''
        train network on given data
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - y: numpy array[int]
            2d numpy array (doc x ner labels) of labels for given data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for training
          - epochs: int (default: 500)
            number of epochs to train for
          - patience: int (default: 10)
            training stops after no improvement in validation score
            for this number of epochs
          - validation_data: tuple (optional)
            tuple of numpy arrays (X,y) representing validation data
          - savebest: boolean (default: False)
            set to True to save the best model based on validation score per epoch
          - filepath: string (optional)
            path to save model if savebest is set to True
        
        outputs:
            None
        '''
    
        if savebest==True and filepath==None:
            raise Exception("Please enter a path to save the network")

        if validation_data:
            validation_size = len(validation_data[0])
        else:
            validation_size = len(X)

        print('training network on %i documents, validation on %i documents' \
              % (len(X), validation_size))

        #track best model for saving
        prevbest = 0
        pat_count = 0

        for ep in range(epochs):

            #shuffle data
            xyz = list(zip(X,y,doc_lens))            
            random.shuffle(xyz)
            X,y,doc_lens = zip(*xyz)
            X = list(X)
            y = list(y)
            doc_lens = list(doc_lens)

            y_pred = []
            y_true = []
            start_time = time.time()

            #train
            for start in range(0,len(X),batch_size):

                #get batch index
                if start+batch_size < len(X):
                    stop = start+batch_size
                else:
                    stop = len(X)

                embeds = np.take(self.embeddings,X[start:stop],0)
                feed_dict = {self.doc_inputs:embeds,
                             self.labels:y[start:stop],
                             self.doc_lens:doc_lens[start:stop],
                             self.doc_idx:self._gen_doc_idx(doc_lens[start:stop]),
                             self.dropout:self.dropout_keep}
                preds,loss,_ = self.sess.run([self.pred,self.loss,self.optimizer],
                              feed_dict=feed_dict)

                #track correct predictions
                for y_pred_,y_true_,l in zip(preds,y[start:stop],doc_lens[start:stop]):
                    y_p = [self.idx2label[l] for l in y_pred_[:l]]
                    y_t = [self.idx2label[l] for l in y_true_[:l]]
                    y_pred.append(y_p)
                    y_true.append(y_t)
                    
                sys.stdout.write("epoch %i, sample %i of %i, loss: %f        \r"\
                                 % (ep+1,stop,len(X),loss))
                sys.stdout.flush()

            #checkpoint after every epoch
            print("\ntraining time: %.2f" % (time.time()-start_time))
            evaluator = Evaluator(y_true, y_pred, tags=['ENT'], loader="list")
            results, results_by_tag = evaluator.evaluate()
            f1 = results['exact']['f1']
            print("epoch %i training f1: %.4f" % (ep+1,f1))

            f1 = self.score(validation_data[0],validation_data[1],
                            validation_data[2],batch_size=batch_size)
            print("epoch %i validation f1: %.4f" % (ep+1,f1))

            #save if performance better than previous best
            if f1 >= prevbest:
                prevbest = f1
                pat_count = 0
                if savebest:
                    self.save(filepath)
            else:
                pat_count += 1
                if pat_count >= patience:
                    break

            #reset timer
            start_time = time.time()

    def predict(self,X,doc_lens,batch_size=128):
    
        '''
        return the predicted labels, flattened labels ignoring padding tokens, 
        and confidence scores for given data
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for inference
            
        outputs:
          - y_pred: numpy_array[int]
            2d numpy array of predicted labels for input data
          - y_pred_flat: list[int]
            flattened list of all predicted labels ignoring padding tokens
          - scores: numpy_array[float]
            flattened list of confidence scores for all predicted labels ignoring padding tokens
        '''
    
        y_pred_flat = []
        y_pred = []
        scores = []
        
        for start in range(0,len(X),batch_size):

            #get batch index
            if start+batch_size < len(X):
                stop = start+batch_size
            else:
                stop = len(X)

            embeds = np.take(self.embeddings,X[start:stop],0)
            feed_dict = {self.doc_inputs:embeds,
                         self.doc_lens:doc_lens[start:stop],
                         self.doc_idx:self._gen_doc_idx(doc_lens[start:stop]),
                         self.dropout:1.0}
            pred,pred_flat,score = self.sess.run([self.pred,self.pred_flat,self.seq_score],
                                   feed_dict=feed_dict)

            score = np.exp(score)
            y_pred.append(pred)
            y_pred_flat.extend(pred_flat)
            scores.extend(score)

            sys.stdout.write("processed %i of %i records        \r" \
                             % (stop,len(X)))
            sys.stdout.flush()

        print()
        y_pred = np.vstack(y_pred)
        return y_pred,y_pred_flat,np.array(scores)

    def score(self,X,y,doc_lens,batch_size=128):
    
        '''
        return the entity-level exact F1 score for given input sequences
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - y: numpy array[int]
            2d numpy array (doc x ner labels) of labels for given data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for inference
            
        outputs:
          - entity-level exact F1 score for given input sequences
        '''
        
        y_preds_,_,_ = self.predict(X,doc_lens,batch_size)
        y_pred = []
        y_true = []
        for y_pred_,y_true_,l in zip(y_preds_,y,doc_lens):
            y_p = [self.idx2label[l] for l in y_pred_[:l]]
            y_t = [self.idx2label[l] for l in y_true_[:l]]
            y_pred.append(y_p)
            y_true.append(y_t)
        
        evaluator = Evaluator(y_true, y_pred, tags=['ENT'], loader="list")
        results, results_by_tag = evaluator.evaluate()
        f1 = results['exact']['f1']
        
        return f1

    def save(self,filename):
    
        '''
        save the model weights to a file
        
        parameters:
          - filepath: string
            path to save model weights
        
        outputs:
            None
        '''
        
        self.saver.save(self.sess,filename)

    def load(self,filename):
    
        '''
        load model weights from a file
        
        parameters:
          - filepath: string
            path from which to load model weights
        
        outputs:
            None
        '''
        
        self.saver.restore(self.sess,filename)


In [ ]:
class tt_bilstm_crf(object):

    '''
    lstm-crf for ner tagging
    
    parameters:
      - embedding_matrix: numpy array[float]
        numpy array of word embeddings
        each row should represent a word embedding
        NOTE: the word index 0 is dropped, so the first row is ignored
      - num_classes: int
        number of output classes
      - max_len: int (default: 50)
        maximum number of input tokens in any sequence
      - rnn_size: int (default: 300)
        number of rnn units in RNN layer
      - dropout_keep: float (default: 0.9)
        dropout keep rate after rnn layer
      - lr: float (default: 1E-4)
        learning rate for adam optimizer
       
    methods:
      - train(X,y,doc_lens,batch_size=128,epochs=25,patience=10,
              validation_data=None,savebest=False,filepath=None)
        train network on given data
      - predict(X,doc_lens,batch_size=128)
        return the predicted labels, flattened labels ignoring padding tokens, 
        and confidence scores for given data
      - score(X,y,doc_lens,batch_size=128)
        return the entity-level exact F1 score for given input sequences
      - save(filepath)
        save the model weights to a file
      - load(filepath)
        load model weights from a file
    '''

    def __init__(self,embedding_matrix,num_classes,max_len=50,rnn_size=300,dropout_keep=0.9,learning_rate=1E-4):
    
        #model params
        self.num_classes = num_classes
        self.embeddings = embedding_matrix.astype(np.float32)
        self.max_len = max_len
        self.rnn_size = rnn_size
        self.dropout_keep = dropout_keep
        self.idx2label = {0:'O',1:'B-ENT',2:'I-ENT'}
        self.lr = learning_rate
        
        #model inputs
        self.doc_inputs = tf.placeholder(tf.float32,shape=[None,max_len,200])
        self.doc_lens = tf.placeholder(tf.int32,shape=[None])
        self.labels = tf.placeholder(tf.int32,shape=[None,max_len])
        self.doc_idx = tf.placeholder(tf.int32,shape=[None,2])
        self.dropout = tf.placeholder(tf.float32)
        rnn_input = tf.nn.dropout(self.doc_inputs,self.dropout)
                      
        #bi-lstm
        with tf.variable_scope('rnn',initializer=tf.contrib.layers.xavier_initializer()):
            [outputs_fw,outputs_bw],_ = tf.nn.bidirectional_dynamic_rnn(
                            LSTMCell(self.rnn_size/2),LSTMCell(self.rnn_size/2),
                            rnn_input,sequence_length=self.doc_lens,dtype=tf.float32)
            outputs = tf.concat((outputs_fw,outputs_bw),2)
            outputs = tf.nn.dropout(outputs,self.dropout)
        
        #linear chain conditional random field
        unary_scores = tf.layers.dense(outputs,self.num_classes,
                       kernel_initializer=tf.contrib.layers.xavier_initializer())
        log_likelihood, self.transition_params = \
                       crf_log_likelihood(unary_scores,self.labels,self.doc_lens)
        self.pred, viterbi_score = crf_decode(unary_scores,self.transition_params,self.doc_lens)
        self.pred_flat = tf.gather_nd(self.pred,self.doc_idx)
        seq_score,_ = crf_log_likelihood(unary_scores,self.pred,self.doc_lens,
                                         self.transition_params)
        self.seq_score = seq_score/tf.cast(self.doc_lens,tf.float32)

        #loss, accuracy, and training functions
        self.loss = tf.reduce_mean(-log_likelihood)
        self.optimizer = tf.train.AdamOptimizer(self.lr,0.9,0.99).minimize(self.loss)
        
        #init ops
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        self.saver = tf.train.Saver()
        self.sess = tf.Session(config=config)
        self.sess.run(tf.global_variables_initializer())

    def _gen_doc_idx(self,doc_lens):
    
        doc_idx = []
        for i,l in enumerate(doc_lens):
            for j in range(l):
                doc_idx.append([i,j])
        doc_idx = np.array(doc_idx)

        return doc_idx
        
    def _flatten_y(self,y,doc_lens):
    
        y_flattened = []
        for i,(doc,l) in enumerate(zip(y,doc_lens)):       
            y_flattened.extend(doc[:l])
            
        return y_flattened
                    
    def train(self,X,y,doc_lens,batch_size=128,epochs=500,patience=20,
              validation_data=None,savebest=False,filepath=None):
    
        '''
        train network on given data
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - y: numpy array[int]
            2d numpy array (doc x ner labels) of labels for given data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for training
          - epochs: int (default: 500)
            number of epochs to train for
          - patience: int (default: 10)
            training stops after no improvement in validation score
            for this number of epochs
          - validation_data: tuple (optional)
            tuple of numpy arrays (X,y) representing validation data
          - savebest: boolean (default: False)
            set to True to save the best model based on validation score per epoch
          - filepath: string (optional)
            path to save model if savebest is set to True
        
        outputs:
            None
        '''
    
        if savebest==True and filepath==None:
            raise Exception("Please enter a path to save the network")

        if validation_data:
            validation_size = len(validation_data[0])
        else:
            validation_size = len(X)

        print('Entered BiLSTM-CRF. Training network on %i documents, validation on %i documents' \
              % (len(X), validation_size))

        #track best model for saving
        prevbest = 0
        pat_count = 0

        for ep in range(epochs):

            #shuffle data
            xyz = list(zip(X,y,doc_lens))            
            random.shuffle(xyz)
            X,y,doc_lens = zip(*xyz)
            X = list(X)
            y = list(y)
            doc_lens = list(doc_lens)

            y_pred = []
            y_true = []
            start_time = time.time()

            #train
            for start in range(0,len(X),batch_size):

                #get batch index
                if start+batch_size < len(X):
                    stop = start+batch_size
                else:
                    stop = len(X)

                embeds = np.take(self.embeddings,X[start:stop],0)
                feed_dict = {self.doc_inputs:embeds,
                             self.labels:y[start:stop],
                             self.doc_lens:doc_lens[start:stop],
                             self.doc_idx:self._gen_doc_idx(doc_lens[start:stop]),
                             self.dropout:self.dropout_keep}
                preds,loss,_ = self.sess.run([self.pred,self.loss,self.optimizer],
                              feed_dict=feed_dict)

                #track correct predictions
                for y_pred_,y_true_,l in zip(preds,y[start:stop],doc_lens[start:stop]):
                    y_p = [self.idx2label[l] for l in y_pred_[:l]]
                    y_t = [self.idx2label[l] for l in y_true_[:l]]
                    y_pred.append(y_p)
                    y_true.append(y_t)
                    
                sys.stdout.write("epoch %i, sample %i of %i, loss: %f        \r"\
                                 % (ep+1,stop,len(X),loss))
                sys.stdout.flush()

            #checkpoint after every epoch
            print("\ntraining time: %.2f" % (time.time()-start_time))
            evaluator = Evaluator(y_true, y_pred, tags=['ENT'], loader="list")
            results, results_by_tag = evaluator.evaluate()
            f1 = results['exact']['f1']
            print("epoch %i training f1: %.4f" % (ep+1,f1))

            f1 = self.score(validation_data[0],validation_data[1],
                            validation_data[2],batch_size=batch_size)
            print("epoch %i validation f1: %.4f" % (ep+1,f1))

            #save if performance better than previous best
            if f1 >= prevbest:
                prevbest = f1
                pat_count = 0
                if savebest:
                    self.save(filepath)
            else:
                pat_count += 1
                if pat_count >= patience:
                    break

            #reset timer
            start_time = time.time()

    def predict(self,X,doc_lens,batch_size=128):
    
        '''
        return the predicted labels, flattened labels ignoring padding tokens, 
        and confidence scores for given data
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for inference
            
        outputs:
          - y_pred: numpy_array[int]
            2d numpy array of predicted labels for input data
          - y_pred_flat: list[int]
            flattened list of all predicted labels ignoring padding tokens
          - scores: numpy_array[float]
            flattened list of confidence scores for all predicted labels ignoring padding tokens
        '''
    
        y_pred_flat = []
        y_pred = []
        scores = []
        
        for start in range(0,len(X),batch_size):

            #get batch index
            if start+batch_size < len(X):
                stop = start+batch_size
            else:
                stop = len(X)

            embeds = np.take(self.embeddings,X[start:stop],0)
            feed_dict = {self.doc_inputs:embeds,
                         self.doc_lens:doc_lens[start:stop],
                         self.doc_idx:self._gen_doc_idx(doc_lens[start:stop]),
                         self.dropout:1.0}
            pred,pred_flat,score = self.sess.run([self.pred,self.pred_flat,self.seq_score],
                                   feed_dict=feed_dict)

            score = np.exp(score)
            y_pred.append(pred)
            y_pred_flat.extend(pred_flat)
            scores.extend(score)

            sys.stdout.write("processed %i of %i records        \r" \
                             % (stop,len(X)))
            sys.stdout.flush()

        print()
        y_pred = np.vstack(y_pred)
        return y_pred,y_pred_flat,np.array(scores)

    def score(self,X,y,doc_lens,batch_size=128):
    
        '''
        return the entity-level exact F1 score for given input sequences
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - y: numpy array[int]
            2d numpy array (doc x ner labels) of labels for given data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for inference
            
        outputs:
          - entity-level exact F1 score for given input sequences
        '''
        
        y_preds_,_,_ = self.predict(X,doc_lens,batch_size)
        y_pred = []
        y_true = []
        for y_pred_,y_true_,l in zip(y_preds_,y,doc_lens):
            y_p = [self.idx2label[l] for l in y_pred_[:l]]
            y_t = [self.idx2label[l] for l in y_true_[:l]]
            y_pred.append(y_p)
            y_true.append(y_t)
        
        evaluator = Evaluator(y_true, y_pred, tags=['ENT'], loader="list")
        results, results_by_tag = evaluator.evaluate()
        f1 = results['exact']['f1']
        
        return f1

    def save(self,filename):
    
        '''
        save the model weights to a file
        
        parameters:
          - filepath: string
            path to save model weights
        
        outputs:
            None
        '''
        
        self.saver.save(self.sess,filename)

    def load(self,filename):
    
        '''
        load model weights from a file
        
        parameters:
          - filepath: string
            path from which to load model weights
        
        outputs:
            None
        '''
        
        self.saver.restore(self.sess,filename)


In [ ]:
class tt_birnn_crf(object):

    '''
    birnn-crf for ner tagging
    
    parameters:
      - embedding_matrix: numpy array[float]
        numpy array of word embeddings
        each row should represent a word embedding
        NOTE: the word index 0 is dropped, so the first row is ignored
      - num_classes: int
        number of output classes
      - max_len: int (default: 50)
        maximum number of input tokens in any sequence
      - rnn_size: int (default: 300)
        number of rnn units in RNN layer
      - dropout_keep: float (default: 0.9)
        dropout keep rate after rnn layer
      - lr: float (default: 1E-4)
        learning rate for adam optimizer
       
    methods:
      - train(X,y,doc_lens,batch_size=128,epochs=25,patience=10,
              validation_data=None,savebest=False,filepath=None)
        train network on given data
      - predict(X,doc_lens,batch_size=128)
        return the predicted labels, flattened labels ignoring padding tokens, 
        and confidence scores for given data
      - score(X,y,doc_lens,batch_size=128)
        return the entity-level exact F1 score for given input sequences
      - save(filepath)
        save the model weights to a file
      - load(filepath)
        load model weights from a file
    '''

    def __init__(self,embedding_matrix,num_classes,max_len=50,rnn_size=300,dropout_keep=0.9,learning_rate=1E-4):
    
        #model params
        self.num_classes = num_classes
        self.embeddings = embedding_matrix.astype(np.float32)
        self.max_len = max_len
        self.rnn_size = rnn_size
        self.dropout_keep = dropout_keep
        self.idx2label = {0:'O',1:'B-ENT',2:'I-ENT'}
        self.lr = learning_rate
        
        #model inputs
        self.doc_inputs = tf.placeholder(tf.float32,shape=[None,max_len,200])
        self.doc_lens = tf.placeholder(tf.int32,shape=[None])
        self.labels = tf.placeholder(tf.int32,shape=[None,max_len])
        self.doc_idx = tf.placeholder(tf.int32,shape=[None,2])
        self.dropout = tf.placeholder(tf.float32)
        rnn_input = tf.nn.dropout(self.doc_inputs,self.dropout)

        #bi-rnn              
        with tf.variable_scope('rnn',initializer=tf.contrib.layers.xavier_initializer()):
            [outputs_fw,outputs_bw],_ = tf.nn.bidirectional_dynamic_rnn(
                            BasicRNNCell(self.rnn_size/2),BasicRNNCell(self.rnn_size/2),
                            rnn_input,sequence_length=self.doc_lens,dtype=tf.float32)
            outputs = tf.concat((outputs_fw,outputs_bw),2)
            outputs = tf.nn.dropout(outputs,self.dropout)
        
        #linear chain conditional random field
        unary_scores = tf.layers.dense(outputs,self.num_classes,
                       kernel_initializer=tf.contrib.layers.xavier_initializer())
        log_likelihood, self.transition_params = \
                       crf_log_likelihood(unary_scores,self.labels,self.doc_lens)
        self.pred, viterbi_score = crf_decode(unary_scores,self.transition_params,self.doc_lens)
        self.pred_flat = tf.gather_nd(self.pred,self.doc_idx)
        seq_score,_ = crf_log_likelihood(unary_scores,self.pred,self.doc_lens,
                                         self.transition_params)
        self.seq_score = seq_score/tf.cast(self.doc_lens,tf.float32)

        #loss, accuracy, and training functions
        self.loss = tf.reduce_mean(-log_likelihood)
        self.optimizer = tf.train.AdamOptimizer(self.lr,0.9,0.99).minimize(self.loss)
        
        #init ops
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        self.saver = tf.train.Saver()
        self.sess = tf.Session(config=config)
        self.sess.run(tf.global_variables_initializer())

    def _gen_doc_idx(self,doc_lens):
    
        doc_idx = []
        for i,l in enumerate(doc_lens):
            for j in range(l):
                doc_idx.append([i,j])
        doc_idx = np.array(doc_idx)

        return doc_idx
        
    def _flatten_y(self,y,doc_lens):
    
        y_flattened = []
        for i,(doc,l) in enumerate(zip(y,doc_lens)):       
            y_flattened.extend(doc[:l])
            
        return y_flattened
                    
    def train(self,X,y,doc_lens,batch_size=128,epochs=500,patience=20,
              validation_data=None,savebest=False,filepath=None):
    
        '''
        train network on given data
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - y: numpy array[int]
            2d numpy array (doc x ner labels) of labels for given data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for training
          - epochs: int (default: 500)
            number of epochs to train for
          - patience: int (default: 10)
            training stops after no improvement in validation score
            for this number of epochs
          - validation_data: tuple (optional)
            tuple of numpy arrays (X,y) representing validation data
          - savebest: boolean (default: False)
            set to True to save the best model based on validation score per epoch
          - filepath: string (optional)
            path to save model if savebest is set to True
        
        outputs:
            None
        '''
    
        if savebest==True and filepath==None:
            raise Exception("Please enter a path to save the network")

        if validation_data:
            validation_size = len(validation_data[0])
        else:
            validation_size = len(X)

        print('Entered BiRNN-CRF. Training network on %i documents, validation on %i documents' \
              % (len(X), validation_size))

        #track best model for saving
        prevbest = 0
        pat_count = 0

        for ep in range(epochs):

            #shuffle data
            xyz = list(zip(X,y,doc_lens))            
            random.shuffle(xyz)
            X,y,doc_lens = zip(*xyz)
            X = list(X)
            y = list(y)
            doc_lens = list(doc_lens)

            y_pred = []
            y_true = []
            start_time = time.time()

            #train
            for start in range(0,len(X),batch_size):

                #get batch index
                if start+batch_size < len(X):
                    stop = start+batch_size
                else:
                    stop = len(X)

                embeds = np.take(self.embeddings,X[start:stop],0)
                feed_dict = {self.doc_inputs:embeds,
                             self.labels:y[start:stop],
                             self.doc_lens:doc_lens[start:stop],
                             self.doc_idx:self._gen_doc_idx(doc_lens[start:stop]),
                             self.dropout:self.dropout_keep}
                preds,loss,_ = self.sess.run([self.pred,self.loss,self.optimizer],
                              feed_dict=feed_dict)

                #track correct predictions
                for y_pred_,y_true_,l in zip(preds,y[start:stop],doc_lens[start:stop]):
                    y_p = [self.idx2label[l] for l in y_pred_[:l]]
                    y_t = [self.idx2label[l] for l in y_true_[:l]]
                    y_pred.append(y_p)
                    y_true.append(y_t)
                    
                sys.stdout.write("epoch %i, sample %i of %i, loss: %f        \r"\
                                 % (ep+1,stop,len(X),loss))
                sys.stdout.flush()

            #checkpoint after every epoch
            print("\ntraining time: %.2f" % (time.time()-start_time))
            evaluator = Evaluator(y_true, y_pred, tags=['ENT'], loader="list")
            results, results_by_tag = evaluator.evaluate()
            f1 = results['exact']['f1']
            print("epoch %i training f1: %.4f" % (ep+1,f1))

            f1 = self.score(validation_data[0],validation_data[1],
                            validation_data[2],batch_size=batch_size)
            print("epoch %i validation f1: %.4f" % (ep+1,f1))

            #save if performance better than previous best
            if f1 >= prevbest:
                prevbest = f1
                pat_count = 0
                if savebest:
                    self.save(filepath)
            else:
                pat_count += 1
                if pat_count >= patience:
                    break

            #reset timer
            start_time = time.time()

    def predict(self,X,doc_lens,batch_size=128):
    
        '''
        return the predicted labels, flattened labels ignoring padding tokens, 
        and confidence scores for given data
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for inference
            
        outputs:
          - y_pred: numpy_array[int]
            2d numpy array of predicted labels for input data
          - y_pred_flat: list[int]
            flattened list of all predicted labels ignoring padding tokens
          - scores: numpy_array[float]
            flattened list of confidence scores for all predicted labels ignoring padding tokens
        '''
    
        y_pred_flat = []
        y_pred = []
        scores = []
        
        for start in range(0,len(X),batch_size):

            #get batch index
            if start+batch_size < len(X):
                stop = start+batch_size
            else:
                stop = len(X)

            embeds = np.take(self.embeddings,X[start:stop],0)
            feed_dict = {self.doc_inputs:embeds,
                         self.doc_lens:doc_lens[start:stop],
                         self.doc_idx:self._gen_doc_idx(doc_lens[start:stop]),
                         self.dropout:1.0}
            pred,pred_flat,score = self.sess.run([self.pred,self.pred_flat,self.seq_score],
                                   feed_dict=feed_dict)

            score = np.exp(score)
            y_pred.append(pred)
            y_pred_flat.extend(pred_flat)
            scores.extend(score)

            sys.stdout.write("processed %i of %i records        \r" \
                             % (stop,len(X)))
            sys.stdout.flush()

        print()
        y_pred = np.vstack(y_pred)
        return y_pred,y_pred_flat,np.array(scores)

    def score(self,X,y,doc_lens,batch_size=128):
    
        '''
        return the entity-level exact F1 score for given input sequences
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - y: numpy array[int]
            2d numpy array (doc x ner labels) of labels for given data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for inference
            
        outputs:
          - entity-level exact F1 score for given input sequences
        '''
        
        y_preds_,_,_ = self.predict(X,doc_lens,batch_size)
        y_pred = []
        y_true = []
        for y_pred_,y_true_,l in zip(y_preds_,y,doc_lens):
            y_p = [self.idx2label[l] for l in y_pred_[:l]]
            y_t = [self.idx2label[l] for l in y_true_[:l]]
            y_pred.append(y_p)
            y_true.append(y_t)
        
        evaluator = Evaluator(y_true, y_pred, tags=['ENT'], loader="list")
        results, results_by_tag = evaluator.evaluate()
        f1 = results['exact']['f1']
        
        return f1

    def save(self,filename):
    
        '''
        save the model weights to a file
        
        parameters:
          - filepath: string
            path to save model weights
        
        outputs:
            None
        '''
        
        self.saver.save(self.sess,filename)

    def load(self,filename):
    
        '''
        load model weights from a file
        
        parameters:
          - filepath: string
            path from which to load model weights
        
        outputs:
            None
        '''
        
        self.saver.restore(self.sess,filename)


In [ ]:
class tt_bigru_crf(object):

    '''
    bigru-crf for ner tagging
    
    parameters:
      - embedding_matrix: numpy array[float]
        numpy array of word embeddings
        each row should represent a word embedding
        NOTE: the word index 0 is dropped, so the first row is ignored
      - num_classes: int
        number of output classes
      - max_len: int (default: 50)
        maximum number of input tokens in any sequence
      - rnn_size: int (default: 300)
        number of rnn units in RNN layer
      - dropout_keep: float (default: 0.9)
        dropout keep rate after rnn layer
      - lr: float (default: 1E-4)
        learning rate for adam optimizer
       
    methods:
      - train(X,y,doc_lens,batch_size=128,epochs=25,patience=10,
              validation_data=None,savebest=False,filepath=None)
        train network on given data
      - predict(X,doc_lens,batch_size=128)
        return the predicted labels, flattened labels ignoring padding tokens, 
        and confidence scores for given data
      - score(X,y,doc_lens,batch_size=128)
        return the entity-level exact F1 score for given input sequences
      - save(filepath)
        save the model weights to a file
      - load(filepath)
        load model weights from a file
    '''

    def __init__(self,embedding_matrix,num_classes,max_len=50,rnn_size=300,dropout_keep=0.9,learning_rate=1E-4):
    
        #model params
        self.num_classes = num_classes
        self.embeddings = embedding_matrix.astype(np.float32)
        self.max_len = max_len
        self.rnn_size = rnn_size
        self.dropout_keep = dropout_keep
        self.idx2label = {0:'O',1:'B-ENT',2:'I-ENT'}
        self.lr = learning_rate
        
        #model inputs
        self.doc_inputs = tf.placeholder(tf.float32,shape=[None,max_len,200])
        self.doc_lens = tf.placeholder(tf.int32,shape=[None])
        self.labels = tf.placeholder(tf.int32,shape=[None,max_len])
        self.doc_idx = tf.placeholder(tf.int32,shape=[None,2])
        self.dropout = tf.placeholder(tf.float32)
        rnn_input = tf.nn.dropout(self.doc_inputs,self.dropout)

        #bi-rnn              
        with tf.variable_scope('rnn',initializer=tf.contrib.layers.xavier_initializer()):
            with tf.variable_scope('forward'):        
              gru_fw_cell = tf.contrib.rnn.GRUCell(self.rnn_size/2)        
              gru_fw_cell = tf.contrib.rnn.DropoutWrapper(gru_fw_cell)    
            with tf.variable_scope('backward'):        
              gru_bw_cell = tf.contrib.rnn.GRUCell(self.rnn_size/2)        
              gru_bw_cell = tf.contrib.rnn.DropoutWrapper(gru_bw_cell)

            [outputs_fw,outputs_bw],_ = tf.nn.bidirectional_dynamic_rnn(
                            gru_fw_cell,gru_bw_cell,
                            rnn_input,sequence_length=self.doc_lens,dtype=tf.float32)
            outputs = tf.concat((outputs_fw,outputs_bw),2)
            outputs = tf.nn.dropout(outputs,self.dropout)
        
        #linear chain conditional random field
        unary_scores = tf.layers.dense(outputs,self.num_classes,
                       kernel_initializer=tf.contrib.layers.xavier_initializer())
        log_likelihood, self.transition_params = \
                       crf_log_likelihood(unary_scores,self.labels,self.doc_lens)
        self.pred, viterbi_score = crf_decode(unary_scores,self.transition_params,self.doc_lens)
        self.pred_flat = tf.gather_nd(self.pred,self.doc_idx)
        seq_score,_ = crf_log_likelihood(unary_scores,self.pred,self.doc_lens,
                                         self.transition_params)
        self.seq_score = seq_score/tf.cast(self.doc_lens,tf.float32)

        #loss, accuracy, and training functions
        self.loss = tf.reduce_mean(-log_likelihood)
        self.optimizer = tf.train.AdamOptimizer(self.lr,0.9,0.99).minimize(self.loss)
        
        #init ops
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        self.saver = tf.train.Saver()
        self.sess = tf.Session(config=config)
        self.sess.run(tf.global_variables_initializer())

    def _gen_doc_idx(self,doc_lens):
    
        doc_idx = []
        for i,l in enumerate(doc_lens):
            for j in range(l):
                doc_idx.append([i,j])
        doc_idx = np.array(doc_idx)

        return doc_idx
        
    def _flatten_y(self,y,doc_lens):
    
        y_flattened = []
        for i,(doc,l) in enumerate(zip(y,doc_lens)):       
            y_flattened.extend(doc[:l])
            
        return y_flattened
                    
    def train(self,X,y,doc_lens,batch_size=128,epochs=500,patience=20,
              validation_data=None,savebest=False,filepath=None):
    
        '''
        train network on given data
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - y: numpy array[int]
            2d numpy array (doc x ner labels) of labels for given data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for training
          - epochs: int (default: 500)
            number of epochs to train for
          - patience: int (default: 10)
            training stops after no improvement in validation score
            for this number of epochs
          - validation_data: tuple (optional)
            tuple of numpy arrays (X,y) representing validation data
          - savebest: boolean (default: False)
            set to True to save the best model based on validation score per epoch
          - filepath: string (optional)
            path to save model if savebest is set to True
        
        outputs:
            None
        '''
    
        if savebest==True and filepath==None:
            raise Exception("Please enter a path to save the network")

        if validation_data:
            validation_size = len(validation_data[0])
        else:
            validation_size = len(X)

        print('Entered BiGRU-CRF. Training network on %i documents, validation on %i documents' \
              % (len(X), validation_size))

        #track best model for saving
        prevbest = 0
        pat_count = 0

        for ep in range(epochs):

            #shuffle data
            xyz = list(zip(X,y,doc_lens))            
            random.shuffle(xyz)
            X,y,doc_lens = zip(*xyz)
            X = list(X)
            y = list(y)
            doc_lens = list(doc_lens)

            y_pred = []
            y_true = []
            start_time = time.time()

            #train
            for start in range(0,len(X),batch_size):

                #get batch index
                if start+batch_size < len(X):
                    stop = start+batch_size
                else:
                    stop = len(X)

                embeds = np.take(self.embeddings,X[start:stop],0)
                feed_dict = {self.doc_inputs:embeds,
                             self.labels:y[start:stop],
                             self.doc_lens:doc_lens[start:stop],
                             self.doc_idx:self._gen_doc_idx(doc_lens[start:stop]),
                             self.dropout:self.dropout_keep}
                preds,loss,_ = self.sess.run([self.pred,self.loss,self.optimizer],
                              feed_dict=feed_dict)

                #track correct predictions
                for y_pred_,y_true_,l in zip(preds,y[start:stop],doc_lens[start:stop]):
                    y_p = [self.idx2label[l] for l in y_pred_[:l]]
                    y_t = [self.idx2label[l] for l in y_true_[:l]]
                    y_pred.append(y_p)
                    y_true.append(y_t)
                    
                sys.stdout.write("epoch %i, sample %i of %i, loss: %f        \r"\
                                 % (ep+1,stop,len(X),loss))
                sys.stdout.flush()

            #checkpoint after every epoch
            print("\ntraining time: %.2f" % (time.time()-start_time))
            evaluator = Evaluator(y_true, y_pred, tags=['ENT'], loader="list")
            results, results_by_tag = evaluator.evaluate()
            f1 = results['exact']['f1']
            print("epoch %i training f1: %.4f" % (ep+1,f1))

            f1 = self.score(validation_data[0],validation_data[1],
                            validation_data[2],batch_size=batch_size)
            print("epoch %i validation f1: %.4f" % (ep+1,f1))

            #save if performance better than previous best
            if f1 >= prevbest:
                prevbest = f1
                pat_count = 0
                if savebest:
                    self.save(filepath)
            else:
                pat_count += 1
                if pat_count >= patience:
                    break

            #reset timer
            start_time = time.time()

    def predict(self,X,doc_lens,batch_size=128):
    
        '''
        return the predicted labels, flattened labels ignoring padding tokens, 
        and confidence scores for given data
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for inference
            
        outputs:
          - y_pred: numpy_array[int]
            2d numpy array of predicted labels for input data
          - y_pred_flat: list[int]
            flattened list of all predicted labels ignoring padding tokens
          - scores: numpy_array[float]
            flattened list of confidence scores for all predicted labels ignoring padding tokens
        '''
    
        y_pred_flat = []
        y_pred = []
        scores = []
        
        for start in range(0,len(X),batch_size):

            #get batch index
            if start+batch_size < len(X):
                stop = start+batch_size
            else:
                stop = len(X)

            embeds = np.take(self.embeddings,X[start:stop],0)
            feed_dict = {self.doc_inputs:embeds,
                         self.doc_lens:doc_lens[start:stop],
                         self.doc_idx:self._gen_doc_idx(doc_lens[start:stop]),
                         self.dropout:1.0}
            pred,pred_flat,score = self.sess.run([self.pred,self.pred_flat,self.seq_score],
                                   feed_dict=feed_dict)

            score = np.exp(score)
            y_pred.append(pred)
            y_pred_flat.extend(pred_flat)
            scores.extend(score)

            sys.stdout.write("processed %i of %i records        \r" \
                             % (stop,len(X)))
            sys.stdout.flush()

        print()
        y_pred = np.vstack(y_pred)
        return y_pred,y_pred_flat,np.array(scores)

    def score(self,X,y,doc_lens,batch_size=128):
    
        '''
        return the entity-level exact F1 score for given input sequences
        
        parameters:
          - X: numpy array[int]
            2d numpy array (doc x word ids) of input data
          - y: numpy array[int]
            2d numpy array (doc x ner labels) of labels for given data
          - doc_lens: list[int]
            true sequence length of each sample
          - batch size: int (default: 128)
            batch size to use for inference
            
        outputs:
          - entity-level exact F1 score for given input sequences
        '''
        
        y_preds_,_,_ = self.predict(X,doc_lens,batch_size)
        y_pred = []
        y_true = []
        for y_pred_,y_true_,l in zip(y_preds_,y,doc_lens):
            y_p = [self.idx2label[l] for l in y_pred_[:l]]
            y_t = [self.idx2label[l] for l in y_true_[:l]]
            y_pred.append(y_p)
            y_true.append(y_t)
        
        evaluator = Evaluator(y_true, y_pred, tags=['ENT'], loader="list")
        results, results_by_tag = evaluator.evaluate()
        f1 = results['exact']['f1']
        
        return f1

    def save(self,filename):
    
        '''
        save the model weights to a file
        
        parameters:
          - filepath: string
            path to save model weights
        
        outputs:
            None
        '''
        
        self.saver.save(self.sess,filename)

    def load(self,filename):
    
        '''
        load model weights from a file
        
        parameters:
          - filepath: string
            path from which to load model weights
        
        outputs:
            None
        '''
        
        self.saver.restore(self.sess,filename)


In [ ]:
# datasets to test on
datasets = [
            #'BC2GM',
            #'BC4CHEMD', 
            #'NCBI-disease',
            's800'
           ]

# supervised dataset sizes to test on
data_sizes = [
              2000
             ]


In [ ]:
pretrain_dataset = 'medmentions'

In [ ]:
batch_size_s = 128
patience_s = 15
holdout_confidence = 0.98

In [ ]:
vocab = np.load('/content/drive/MyDrive/Pubmed/vocab.npy').astype(np.int16)

Asymmetric tri-training for aforementioned datasets of specified labeled training sizes

In [ ]:
for dataset in datasets:

    # iterate over each data size setting
    for data_size in data_sizes:

        print('Training in the dataset loop',dataset,data_size)
        
        # load data
        X = np.load('/content/drive/MyDrive/Pubmed/%s_X_train.npy' % dataset).astype(np.int16)
        y = np.load('/content/drive/MyDrive/Pubmed/%s_y_train.npy' % dataset).astype(np.int16)
        max_len = 50
        num_classes = np.max(y) + 1
        
        with open('/content/drive/MyDrive/Pubmed/%s_senlens_train.pkl' % dataset,'rb') as f:
            doc_len = pickle.load(f)
        doc_len = [l if l <=max_len else max_len for l in doc_len]
        
        # load conversion dictionaries
        label2idx = {'O':0, 'B':1, 'I': 2}
        idx2label = {0:'O',1:'B-ENT',2:'I-ENT'}
            
        # train val split
        start_size = data_size
        train_size = int(0.8 * data_size)
        X_start = X[:start_size]
        y_start = y[:start_size]
        doc_len_start = np.array(doc_len[:start_size])
        X_holdout = X[start_size:]
        y_holdout = y[start_size:]
        doc_len_holdout = np.array(doc_len[start_size:])
        
        X_train = X_start[:train_size]
        X_val = X_start[train_size:]
        y_train = y_start[:train_size]
        y_val = y_start[train_size:]
        doc_len_train = np.array(doc_len_start[:train_size])
        doc_len_val = np.array(doc_len_start[train_size:])
        
        # load test data
        X_test = np.load('/content/drive/MyDrive/Pubmed/%s_X_test.npy' % dataset).astype(np.int16)
        y_test = np.load('/content/drive/MyDrive/Pubmed/%s_y_test.npy' % dataset).astype(np.int16)
        with open('/content/drive/MyDrive/Pubmed/%s_senlens_test.pkl' % dataset,'rb') as f:
            doc_len_test = pickle.load(f)
        doc_len_test = [l if l <=max_len else max_len for l in doc_len_test]
        y_true = []
        for y_true_,l in zip(y_test,doc_len_test):
            y = [idx2label[l] for l in y_true_[:l]]
            y_true.append(y)
        y_true_val = []
        for y_true_,l in zip(y_val,doc_len_val):
            y = [idx2label[l] for l in y_true_[:l]]
            y_true_val.append(y)
        

        i = 0
        training_sizes = []
        filter_size = np.inf
        
        # loop through holdout set until no new samples are added
        while filter_size > 0:
                                
            num_samples = len(X_train)
            training_sizes.append(num_samples)
            print('train size:',num_samples)
            
            # train model
            tf.reset_default_graph()
            model1 = tt_bilstm_crf(vocab, num_classes, max_len)
            tf.reset_default_graph()
            model2 = tt_birnn_crf(vocab, num_classes, max_len)
            tf.reset_default_graph()
            model3 = tt_bigru_crf(vocab, num_classes, max_len)
            
            
            model1.train(X_train,y_train,doc_len_train,
                        batch_size=batch_size_s,patience=patience_s,
                        validation_data=(X_val,y_val,doc_len_val),
                        savebest=True,filepath='/content/drive/MyDrive/Pubmed/SavedModels/%s_tt_lstm_crf_%i_%i.ckpt' % (dataset,train_size,i))
            
            model2.train(X_train,y_train,doc_len_train,
                        batch_size=batch_size_s,patience=patience_s,
                        validation_data=(X_val,y_val,doc_len_val),
                        savebest=True,filepath='/content/drive/MyDrive/Pubmed/SavedModels/%s_tt_rnn_crf_%i_%i.ckpt' % (dataset,train_size,i))
            
            
            # add high conf from holdout to test
            model1.load('/content/drive/MyDrive/Pubmed/SavedModels/%s_tt_lstm_crf_%i_%i.ckpt' % (dataset,train_size,i))
            model2.load('/content/drive/MyDrive/Pubmed/SavedModels/%s_tt_rnn_crf_%i_%i.ckpt' % (dataset,train_size,i))
            
            # Procedure for adding samples to training set via M1-M2 combination
            y_pred_lstm,_,scores_lstm = model1.predict(X_holdout,doc_len_holdout)
            #print('y_pred_lstm: ', y_pred_lstm)
            #print('scores lstm: ', scores_lstm)
            
            y_pred_rnn,_,scores_rnn = model2.predict(X_holdout,doc_len_holdout)
            #print('y_pred_rnn: ', y_pred_rnn)
            #print('scores rnn: ', scores_rnn)

            print('Before M1-M2 iteration, doc_len_holdout array size:', doc_len_holdout.size)
            print('Before M1-M2 iteration, X_train array:', X_train.size)

            val1 = []
            # For adding samples to training set through predictions of M1 and M2 on set S
            for x in range(y_pred_lstm.shape[0]):
              if(np.array_equiv(y_pred_lstm[x], y_pred_rnn[x]) and (min(scores_lstm[x], scores_rnn[x])>=holdout_confidence)):
                val1.insert(x, True)
              else:
                val1.insert(x, False)   
            
            val_size1 = np.sum(val1)
            print('Val size 1: ', val_size1)
            print('For M1-M2 combination, iteration %i: adding %i samples from holdout to train' % (i+1, val_size1))
            X_train = np.vstack((X_train,X_holdout[val1]))
            print('Printing X_train after adding from holdout', X_train.shape)
            y_train = np.vstack((y_train,y_pred_lstm[val1]))
            print('Printing y_train after adding from holdout', y_train.shape)
            doc_len_train = np.hstack((doc_len_train,doc_len_holdout[val1]))
            print('Printing doc_len_train after adding from holdout', doc_len_train.shape)
            val_inv1 = np.invert(val1)
            invert_size1 = np.sum(val_inv1)
            print('Printing inverted val size: ', invert_size1)
            X_holdout = X_holdout[val_inv1]
            print('Printing X_holdout after removing from holdout', X_holdout.shape)
            doc_len_holdout = doc_len_holdout[val_inv1]
            print('Printing doc_len_holdout after removing from holdout', doc_len_holdout.shape)

            print('After M1-M2 iteration, doc_len_holdout array:', doc_len_holdout.size)
            print('After M1-M2 iteration, X_train array:', X_train.size)
            

            model3.train(X_train,y_train,doc_len_train,
                        batch_size=batch_size_s,patience=patience_s,
                        validation_data=(X_val,y_val,doc_len_val),
                        savebest=True,filepath='/content/drive/MyDrive/Pubmed/SavedModels/%s_tt_gru_crf_%i_%i.ckpt' % (dataset,train_size,i))
            model3.load('/content/drive/MyDrive/Pubmed/SavedModels/%s_tt_gru_crf_%i_%i.ckpt' % (dataset,train_size,i))
            
            # Procedure for adding samples to training set via M1-M3 combination
            y_pred_gru,_,scores_gru = model3.predict(X_holdout,doc_len_holdout)
            #print('y_pred_gru: ', y_pred_gru.shape)
            #print('scores gru: ', scores_gru.shape)

            y_pred_lstm,_,scores_lstm = model1.predict(X_holdout,doc_len_holdout)
            #print('y_pred_lstm: ', y_pred_lstm.shape)
            #print('scores_lstm: ', scores_lstm.shape)

            print('Before M1-M3 iteration, doc_len_holdout array:', doc_len_holdout.size)
            print('Before M1-M3 iteration, X_train array:', X_train.size)

            val2 = []
            for x in range(y_pred_gru.shape[0]):
              if(np.array_equiv(y_pred_lstm[x], y_pred_gru[x]) and (min(scores_lstm[x], scores_gru[x])>=holdout_confidence)):
                val2.insert(x, True)
              else:
                val2.insert(x, False)
                
            val_size2 = np.sum(val2)
            print('Val size 2: ', val_size2)
            print('iteration %i: adding %i samples from holdout to train' % (i+1, val_size2))
            X_train = np.vstack((X_train,X_holdout[val2]))
            print('Printing X_train after adding from holdout', X_train.shape)
            y_train = np.vstack((y_train,y_pred_gru[val2]))
            print('Printing y_train after adding from holdout', y_train.shape)
            doc_len_train = np.hstack((doc_len_train,doc_len_holdout[val2]))
            print('Printing doc_len_train after adding from holdout', doc_len_train.shape)
            val_inv2 = np.invert(val2)
            invert_size2 = np.sum(val_inv2)
            print('Printing inverted val size: ', invert_size2)
            X_holdout = X_holdout[val_inv2]
            print('Printing X_holdout after adding from holdout', X_holdout.shape)
            doc_len_holdout = doc_len_holdout[val_inv2]
            print('Printing doc_len_holdout after adding from holdout', doc_len_holdout.shape)    

            print('After M1-M3 iteration, doc_len_holdout array:', doc_len_holdout.size)
            print('After M1-M3 iteration, X_train array:', X_train.size)  
            
            # Procedure for adding samples to training set via M2-M3 combination
            y_pred_gru,_,scores_gru = model3.predict(X_holdout,doc_len_holdout)
            print('y_pred_gru: ', y_pred_gru.shape)
            print('scores gru: ', scores_gru.shape)

            y_pred_rnn,_,scores_rnn = model2.predict(X_holdout,doc_len_holdout)
            print('y_pred_rnn: ', y_pred_rnn.shape)
            print('scores_rnn: ', scores_rnn.shape)

            print('Before M2-M3 iteration, doc_len_holdout array:', doc_len_holdout.size)
            print('Before M2-M3 iteration, X_train array:', X_train.size)
            
            val3 = []
            for x in range(y_pred_gru.shape[0]):
              if(np.array_equiv(y_pred_rnn[x], y_pred_gru[x]) and (min(scores_rnn[x], scores_gru[x])>=holdout_confidence)):
                val3.insert(x, True)
              else:
                val3.insert(x, False)
            
            val_size3 = np.sum(val3)
            print('Val size-3: ', val_size3)
            print('iteration %i: adding %i samples from holdout to train' % (i+1, val_size3))
            X_train = np.vstack((X_train,X_holdout[val3]))
            print('Printing X_train after adding from holdout', X_train.shape)
            y_train = np.vstack((y_train,y_pred_gru[val3]))
            print('Printing y_train after adding from holdout', y_train.shape)
            doc_len_train = np.hstack((doc_len_train,doc_len_holdout[val3]))
            print('Printing doc_len_train after adding from holdout', doc_len_train.shape)
            val_inv3 = np.invert(val3)
            invert_size3 = np.sum(val_inv3)
            print('Printing inverted val size: ', invert_size3)
            X_holdout = X_holdout[val_inv3]
            print('Printing X_holdout after adding from holdout', X_holdout.shape)
            doc_len_holdout = doc_len_holdout[val_inv3]
            print('Printing doc_len_holdout after adding from holdout', doc_len_holdout.shape)
            
            print('After M2-M3 iteration, doc_len_holdout array:', doc_len_holdout.size)
            print('After M2-M3 iteration, X_train array:', X_train.size)

            filter_size = val_size1 + val_size2 + val_size3
            print('Filter size for this iteration: ', filter_size)

            # evaluate on test set for M1
            y_preds_l_,_,_ = model1.predict(X_test,doc_len_test)
            y_predl = []
            for y_pred1_,l in zip(y_preds_l_,doc_len_test):
                y = [idx2label[l] for l in y_pred1_[:l]]
                y_predl.append(y)
            
            evaluator1 = Evaluator(y_true, y_predl, tags=['ENT'], loader="list")
            results1, results_by_tag1 = evaluator1.evaluate()
            
            precision1 = results1['exact']['precision']
            recall1 = results1['exact']['recall']
            f1_1 = results1['exact']['f1']

            print('Test set evaluation for BiLSTM CRF iteration %i results on %i samples' % (i+1,num_samples))
            print('precision:',precision1)
            print('recall:',recall1)
            print('f1:',f1_1)
            print()

            # evaluate on test set for M2
            y_preds_r_,_,_ = model2.predict(X_test,doc_len_test)
            y_predr = []
            for y_pred2_,l in zip(y_preds_r_,doc_len_test):
                y = [idx2label[l] for l in y_pred2_[:l]]
                y_predr.append(y)
            
            evaluator2 = Evaluator(y_true, y_predr, tags=['ENT'], loader="list")
            results2, results_by_tag2 = evaluator2.evaluate()
            
            precision2 = results2['exact']['precision']
            recall2 = results2['exact']['recall']
            f1_2 = results2['exact']['f1']

            print('Test set evaluation for BiRNN CRF iteration %i results on %i samples' % (i+1,num_samples))
            print('precision:',precision2)
            print('recall:',recall2)
            print('f1:',f1_2)
            print()

            # evaluate on test set for M3
            y_preds_g_,_,_ = model3.predict(X_test,doc_len_test)
            y_predg = []
            for y_pred3_,l in zip(y_preds_g_,doc_len_test):
                y = [idx2label[l] for l in y_pred3_[:l]]
                y_predg.append(y)
            
            evaluator3 = Evaluator(y_true, y_predg, tags=['ENT'], loader="list")
            results3, results_by_tag3 = evaluator3.evaluate()
            
            precision3 = results3['exact']['precision']
            recall3 = results3['exact']['recall']
            f1_3 = results3['exact']['f1']

            print('Test set evaluation for BiGRU CRF iteration %i results on %i samples' % (i+1,num_samples))
            print('precision:',precision3)
            print('recall:',recall3)
            print('f1:',f1_3)
            print()
            
            i += 1

            # track size of training set after each iteration of semi-supervised training
            with open('/content/drive/MyDrive/Pubmed/Asymmetric/%s_tritrain_datasize_%i.pkl' % (dataset,train_size),'wb') as f:
              pickle.dump(training_sizes,f)
            np.save("/content/drive/MyDrive/Pubmed/Asymmetric/%s_X_train_%i_it_%i.npy" % (dataset,data_size,i+1),X_train)
            np.save("/content/drive/MyDrive/Pubmed/Asymmetric/%s_y_train_%i_it_%i.npy" % (dataset,data_size,i+1),y_train)
            np.save("/content/drive/MyDrive/Pubmed/Asymmetric/%s_X_val_%i_it_%i.npy" % (dataset,data_size,i+1),X_val)
            np.save("/content/drive/MyDrive/Pubmed/Asymmetric/%s_y_val_%i_it_%i.npy" % (dataset,data_size,i+1),y_val)
            with open('/content/drive/MyDrive/Pubmed/Asymmetric/%s_senlens_%i_it_%i.pkl' % (dataset,data_size,i+1),'wb') as f:
              pickle.dump(doc_len_train,f)
            with open('/content/drive/MyDrive/Pubmed/Asymmetric/%s_senlens_val_%i_it_%i.pkl' % (dataset,data_size,i+1),'wb') as f:
              pickle.dump(doc_len_val,f)
        

Training in the dataset loop BC2GM 2000
train size: 1600
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2022-04-28 05:56:45 tensorflow WARNING: From <ipython-input-7-8b90d78dcfaa>:54: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



2022-04-28 05:56:45 tensorflow WARNING: 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


...

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Pubmed/SavedModels/BC2GM_tt_rnn_crf_1600_23.ckpt


2022-04-29 05:10:48 tensorflow INFO: Restoring parameters from /content/drive/MyDrive/Pubmed/SavedModels/BC2GM_tt_rnn_crf_1600_23.ckpt




Before M1-M2 iteration, doc_len_holdout array size: 2769
Before M1-M2 iteration, X_train array: 470250
Val size 1:  4
For M1-M2 combination, iteration 24: adding 4 samples from holdout to train
Printing X_train after adding from holdout (9409, 50)
Printing y_train after adding from holdout (9409, 50)
Printing doc_len_train after adding from holdout (9409,)
Printing inverted val size:  2765
Printing X_holdout after removing from holdout (2765, 50)
Printing doc_len_holdout after removing from holdout (2765,)
After M1-M2 iteration, doc_len_holdout array: 2765
After M1-M2 iteration, X_train array: 470450
Entered BiGRU-CRF. Training network on 9409 documents, validation on 400 documents

training time: 12.74
epoch 1 training f1: 0.0079

epoch 1 validation f1: 0.0057

training time: 11.93
epoch 2 training f1: 0.0023

epoch 2 validation f1: 0.0000

training time: 12.08
epoch 3 training f1: 0.0036

epoch 3 validation f1: 0.0000

training time: 11.79
epoch 4 training f1: 0.0106

epoch 4 valid

2022-04-29 05:28:59 tensorflow INFO: Restoring parameters from /content/drive/MyDrive/Pubmed/SavedModels/BC2GM_tt_gru_crf_1600_23.ckpt




Before M1-M3 iteration, doc_len_holdout array: 2765
Before M1-M3 iteration, X_train array: 470450
Val size 2:  4
iteration 24: adding 4 samples from holdout to train
Printing X_train after adding from holdout (9413, 50)
Printing y_train after adding from holdout (9413, 50)
Printing doc_len_train after adding from holdout (9413,)
Printing inverted val size:  2761
Printing X_holdout after adding from holdout (2761, 50)
Printing doc_len_holdout after adding from holdout (2761,)
After M1-M3 iteration, doc_len_holdout array: 2761
After M1-M3 iteration, X_train array: 470650

y_pred_gru:  (2761, 50)
scores gru:  (2761,)

y_pred_rnn:  (2761, 50)
scores_rnn:  (2761,)
Before M2-M3 iteration, doc_len_holdout array: 2761
Before M2-M3 iteration, X_train array: 470650
Val size-3:  2
iteration 24: adding 2 samples from holdout to train
Printing X_train after adding from holdout (9415, 50)
Printing y_train after adding from holdout (9415, 50)
Printing doc_len_train after adding from holdout (9415,)

Finetuning the pretrained model on obtained training data

In [ ]:
batch_size_as = 128
patience_as = 15

In [ ]:
for dataset in datasets:

    # iterate over each data size setting
    for data_size in data_sizes:

        print('Training in the dataset loop',dataset,data_size)
        
        # load training data
        X_train = np.load("/content/drive/MyDrive/Pubmed/Asymmetric/%s_X_train_%i.npy" % (dataset,data_size)).astype(np.int16)
        y_train = np.load("/content/drive/MyDrive/Pubmed/Asymmetric/%s_y_train_%i.npy" % (dataset,data_size)).astype(np.int16)
        max_len = 50
        num_classes = np.max(y_train) + 1
      
        with open('/content/drive/MyDrive/Pubmed/Asymmetric/%s_senlens_%i.pkl' % (dataset,data_size),'rb') as f:
            doc_len_train = pickle.load(f)
        doc_len_train = [l if l <=max_len else max_len for l in doc_len_train]

        # load training data
        X_val = np.load("/content/drive/MyDrive/Pubmed/Asymmetric/%s_X_val_%i.npy" % (dataset,data_size)).astype(np.int16)
        y_val = np.load("/content/drive/MyDrive/Pubmed/Asymmetric/%s_y_val_%i.npy" % (dataset,data_size)).astype(np.int16)
       
        with open('/content/drive/MyDrive/Pubmed/Asymmetric/%s_senlens_val_%i.pkl' % (dataset,data_size),'rb') as f:
            doc_len_val = pickle.load(f)
        doc_len_val = [l if l <=max_len else max_len for l in doc_len_val]
        
        # load conversion dictionaries
        label2idx = {'O':0, 'B':1, 'I': 2}
        idx2label = {0:'O',1:'B-ENT',2:'I-ENT'}
            
        # load test data
        X_test = np.load('/content/drive/MyDrive/Pubmed/%s_X_test.npy' % dataset).astype(np.int16)
        y_test = np.load('/content/drive/MyDrive/Pubmed/%s_y_test.npy' % dataset).astype(np.int16)
        with open('/content/drive/MyDrive/Pubmed/%s_senlens_test.pkl' % dataset,'rb') as f:
            doc_len_test = pickle.load(f)
        doc_len_test = [l if l <=max_len else max_len for l in doc_len_test]
        y_true = []
        for y_true_,l in zip(y_test,doc_len_test):
            y = [idx2label[l] for l in y_true_[:l]]
            y_true.append(y)
        y_true_val = []
        for y_true_,l in zip(y_val,doc_len_val):
            y = [idx2label[l] for l in y_true_[:l]]
            y_true_val.append(y)
        
        tf.reset_default_graph()
        model = lstm_crf(vocab,num_classes,max_len)
        
        # Here load the pretrained model and train it on the new training set obtained after semi-supervised learning
        model.load('/content/drive/MyDrive/Pubmed/PretrainModel/lstm_crf_%s.ckpt' % pretrain_dataset)

        model.train(X_train,y_train,doc_len_train,
                    batch_size=batch_size_as,patience=patience_as,
                    validation_data=(X_val,y_val,doc_len_val),
                    savebest=True,filepath='/content/drive/MyDrive/Pubmed/FinetuneAsymmetric/%s_finetune_%s_%i.ckpt' % (dataset,pretrain_dataset,data_size))
        
        # Finally evaluate on test set
        model.load('/content/drive/MyDrive/Pubmed/FinetuneAsymmetric/%s_finetune_%s_%i.ckpt' % (dataset,pretrain_dataset,data_size))
        y_preds_,_,_ = model.predict(X_test,doc_len_test)
        y_pred = []
        for y_pred_,l in zip(y_preds_,doc_len_test):
            y = [idx2label[l] for l in y_pred_[:l]]
            y_pred.append(y)
        
        evaluator = Evaluator(y_true, y_pred, tags=['ENT'], loader="list")
        results, results_by_tag = evaluator.evaluate()
        
        exact_p = results['exact']['precision']
        exact_r = results['exact']['recall']
        exact_f = results['exact']['f1']
        partial_p = results['partial']['precision']
        partial_r = results['partial']['recall']
        partial_f = results['partial']['f1']
    
        print(dataset, data_size)
        print('exact p: %.4f' % exact_p)
        print('exact r: %.4f' % exact_r)
        print('exact f: %.4f' % exact_f)
        print('partial p: %.4f' % partial_p)
        print('partial r: %.4f' % partial_r)
        print('partial f: %.4f' % partial_f)
        print()



Training in the dataset loop s800 3000
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Pubmed/PretrainModel/lstm_crf_medmentions.ckpt


2022-05-01 10:50:45 tensorflow INFO: Restoring parameters from /content/drive/MyDrive/Pubmed/PretrainModel/lstm_crf_medmentions.ckpt


training network on 4684 documents, validation on 600 documents

training time: 11.66
epoch 1 training f1: 0.0260

epoch 1 validation f1: 0.0336

training time: 10.95
epoch 2 training f1: 0.0479

epoch 2 validation f1: 0.1163

training time: 10.84
epoch 3 training f1: 0.0969

epoch 3 validation f1: 0.1390

training time: 10.84
epoch 4 training f1: 0.1262

epoch 4 validation f1: 0.1603

training time: 10.74
epoch 5 training f1: 0.1345

epoch 5 validation f1: 0.1993

training time: 10.88
epoch 6 training f1: 0.1503

epoch 6 validation f1: 0.1955

training time: 11.13
epoch 7 training f1: 0.1676

epoch 7 validation f1: 0.2109

training time: 10.96
epoch 8 training f1: 0.1818

epoch 8 validation f1: 0.2132

training time: 10.96
epoch 9 training f1: 0.1975

epoch 9 validation f1: 0.2340

training time: 11.14
epoch 10 training f1: 0.2159

epoch 10 validation f1: 0.2612

training time: 10.99
epoch 11 training f1: 0.2469

epoch 11 validation f1: 0.2609

training time: 10.80
epoch 12 training f

2022-05-01 10:59:51 tensorflow INFO: Restoring parameters from /content/drive/MyDrive/Pubmed/FinetuneAsymmetric/s800_finetune_medmentions_3000.ckpt



s800 3000
exact p: 0.4538
exact r: 0.2227
exact f: 0.2987
partial p: 0.6005
partial r: 0.2947
partial f: 0.3953

